In [ ]:
import pandas as pd

from src.params import parse_arguments
from src.utils import build_model_from_args, set_seed, get_device
from src.data import get_loader

args = parse_arguments()
set_seed(args.seed)
args.device=get_device(force_cpu=True)
print(args)
print(args.device)

In [2]:
## --- change args here
args.model_name_extra = ''

In [16]:
df = pd.read_csv('/proj/berzelius-2024-156/users/x_yxiao/AI-PET/data/demo_masked.csv', index_col=0)
print(df['visual_read'].value_counts())
print(df['gender'].value_counts())
df

visual_read
0.0    6332
1.0    4018
2.0       4
Name: count, dtype: int64
gender
2    5243
1    5107
3       4
Name: count, dtype: int64


,ID,pet_path,ScanDate,tracer,site,tracer_selected,CL,visual_read,visual_read_confidence,age,gender,ethnicity,education,diagnosis,AD_pathology,APOE
0,0,0,NaN,NaN,20644,2.0,70.0,0.0,3.0,73,2,1,7,11.0,7.0,0
1,1,0,NaN,NaN,17612,1.0,119.5,0.0,3.0,73,2,1,7,13.0,9.0,0
2,2,0,NaN,NaN,17707,2.0,61.7,0.0,3.0,68,1,1,5,12.0,8.0,0
3,3,0,NaN,NaN,21367,2.0,24.7,0.0,3.0,83,2,1,5,26.0,5.0,0
4,4,0,NaN,NaN,17652,1.0,114.7,0.0,3.0,81,1,1,6,14.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10349,10349,0,NaN,NaN,20753,2.0,62.8,0.0,1.0,86,2,1,4,13.0,6.0,0
10350,10350,0,NaN,NaN,17601,1.0,-4.1,1.0,3.0,73,2,1,3,11.0,9.0,0
10351,10351,0,NaN,NaN,20267,2.0,-30.4,1.0,2.0,77,2,1,6,13.0,5.0,0
10352,10352,0,NaN,NaN,17636,3.0,58.9,0.0,3.0,70,1,1,7,11.0,3.0,0


In [17]:
df_new = df[(df['visual_read']!=2) & (df['gender']!=3)]
df_new["visual_read"] = 1 - df_new["visual_read"]
print(df_new['visual_read'].value_counts())
print(df_new['gender'].value_counts())
df_new

visual_read
1.0    6329
0.0    4017
Name: count, dtype: int64
gender
2    5241
1    5105
Name: count, dtype: int64


/tmp/ipykernel_2561369/157700529.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["visual_read"] = 1 - df_new["visual_read"]


,ID,pet_path,ScanDate,tracer,site,tracer_selected,CL,visual_read,visual_read_confidence,age,gender,ethnicity,education,diagnosis,AD_pathology,APOE
0,0,0,NaN,NaN,20644,2.0,70.0,1.0,3.0,73,2,1,7,11.0,7.0,0
1,1,0,NaN,NaN,17612,1.0,119.5,1.0,3.0,73,2,1,7,13.0,9.0,0
2,2,0,NaN,NaN,17707,2.0,61.7,1.0,3.0,68,1,1,5,12.0,8.0,0
3,3,0,NaN,NaN,21367,2.0,24.7,1.0,3.0,83,2,1,5,26.0,5.0,0
4,4,0,NaN,NaN,17652,1.0,114.7,1.0,3.0,81,1,1,6,14.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10349,10349,0,NaN,NaN,20753,2.0,62.8,1.0,1.0,86,2,1,4,13.0,6.0,0
10350,10350,0,NaN,NaN,17601,1.0,-4.1,0.0,3.0,73,2,1,3,11.0,9.0,0
10351,10351,0,NaN,NaN,20267,2.0,-30.4,0.0,2.0,77,2,1,6,13.0,5.0,0
10352,10352,0,NaN,NaN,17636,3.0,58.9,1.0,3.0,70,1,1,7,11.0,3.0,0


In [18]:
df_new.to_csv('/proj/berzelius-2024-156/users/x_yxiao/AI-PET/data/demo_masked_fixed.csv')

# GradCam

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from monai.visualize.class_activation_maps import GradCAM

def grad_cam(model, x, # [1, C, D, H, W]
             target_layer: str | None = None,      # auto-pick last Conv3d if None
             class_idx: int | None = None,         # auto from model output if None
             upsample_mode: str = "trilinear", align_corners: bool = False, 
             normalize: bool = True) -> np.ndarray: # min-max to [0,1]
    """
    Compute Grad-CAM (or Grad-CAM++) for a 3D model that outputs logits.
    Returns a NumPy array of shape (D, H, W).
    """
    assert x.ndim == 5 and x.size(0) == 1, "x must be a single 3D volume: [1, C, D, H, W]"
    model.eval()

    # --- pick target layer: last Conv3d if not provided
    if target_layer is None:
        last = None
        for name, m in model.named_modules():
            if isinstance(m, nn.Conv3d):
                last = name
        if last is None: raise ValueError("No Conv3d layer found; specify target_layer explicitly.")
        target_layer = last
    #print(f"Using target layer: {target_layer}")

    # --- forward once to get logits and decide class_idx if needed
    if class_idx is None:
        with torch.inference_mode():
            logits = model(x)  # expected shapes: [B,1] or [B,2] (or [B,C])
        if logits.ndim == 1: logits = logits.unsqueeze(1)    # [B] -> make it [B,1]
        n_classes = logits.shape[1]
        class_idx = 0 if n_classes == 1 else int(torch.argmin(logits, dim=1).item())  # predicted class # argmin because the pos and neg flag is revert in trainnig
    else: # sanity check
        if not (0 <= int(class_idx) < n_classes):
            raise ValueError(f"class_idx {class_idx} out of range 0..{n_classes-1}")
    
    cam = GradCAM(nn_module=model, target_layers=target_layer)
    # --- compute CAM (disable AMP for stable grads)
    with torch.amp.autocast(device_type="cuda", enabled=False):
        cam_map = cam(x, class_idx=int(class_idx))   # [B, 1, d, h, w]
    cam_map = cam_map.detach()

    # --- upsample to input spatial size if needed
    in_spatial = x.shape[-3:]
    cam_spatial = cam_map.shape[-3:]
    if cam_spatial != in_spatial:
        print('Upsample to input space')
        cam_map = F.interpolate(cam_map, size=in_spatial, mode=upsample_mode, align_corners=align_corners)

    cam_np = cam_map[0, 0].cpu().float().numpy()  # (D, H, W)

    # --- normalize to [0,1] if requested
    if normalize:
        vmin, vmax = np.nanmin(cam_np), np.nanmax(cam_np)
        if np.isfinite(vmin) and np.isfinite(vmax) and vmax > vmin:
            cam_np = (cam_np - vmin) / (vmax - vmin)
        else:
            cam_np = np.zeros_like(cam_np, dtype=np.float32)

    return cam_np, class_idx, logits.detach().cpu().numpy() # (D, H, W)


In [12]:
import matplotlib.pyplot as plt
def show_cam(x, cam_np):
    img_np = x.detach().cpu().numpy()[0, 0]     # (D,H,W)
    m = (cam_np - cam_np.min()) / (np.ptp(cam_np) + 1e-8)
    z = img_np.shape[0] // 2

    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1); plt.imshow(img_np[z], cmap="gray"); plt.title("Image")
    plt.subplot(1,2,2); plt.imshow(img_np[z], cmap="gray"); plt.imshow(m[z], alpha=0.5); plt.title("Grad-CAM")
    plt.tight_layout(); plt.show()

In [ ]:
import os
import torch
import numpy as np
import nibabel as nib

@torch.no_grad()
def gradcam_to_nifti(cam_np, out_path: str, ref_nii_path: str,):  # use affine/header from this NIfTI (recommended)
    """
    Save a CAM volume (D,H,W) as NIfTI using affine/header from a reference image.
    """
    if not (isinstance(cam_np, np.ndarray) and cam_np.ndim == 3):
        raise ValueError("cam_np must be a 3D NumPy array (D,H,W).")
    if not (ref_nii_path and os.path.exists(ref_nii_path)):
        raise ValueError("ref_nii_path must exist and point to a valid NIfTI file.")

    ref_img = nib.load(ref_nii_path)
    affine = ref_img.affine
    hdr = ref_img.header.copy()
    hdr.set_data_shape(cam_np.shape)

    out_img = nib.Nifti1Image(cam_np.astype(np.float32, copy=False), affine, header=hdr)
    nib.save(out_img, out_path)
    print(f"Saved image to {out_path}")

### IDEAS

#### Visual reads

In [6]:
import os
test_set = os.path.join(args.best_model_folder,'Hold-out_testing-set.csv')
print(test_set)
df = pd.read_csv(test_set, index_col=0)
tfm = args.input_path

dl_va = get_loader(df, tfm, args, batch_size=max(1, args.batch_size // 2), augment=False, shuffle=False)

/proj/berzelius-2024-156/users/x_yxiao/AI-PET/results/CNN3D_visual_read_2split80-20_stratify-visual_read,site_IDEAS_Inten_Norm_20251004_022211/Hold-out_testing-set.csv


In [7]:
import torch

targets_list = [t.strip() for t in args.targets.split(",") if t.strip()]
n_classes = int(df["visual_read"].dropna().nunique()) if 'visual_read' in targets_list else None
model = build_model_from_args(args, device=args.device, n_classes=n_classes)
sd = torch.load(os.path.join(args.best_model_folder, 'outer-test/checkpoints', 'best.pt'), map_location=args.device, weights_only=True)
state_dict = sd.get("model", sd) if isinstance(sd, dict) else sd
model.load_state_dict(state_dict, strict=False)

in_channels: 1, widths: (32, 64, 128, 256), pool_every: 1, dropout: 0.3, norm: batch, num_classes: 2
CNN3D(
  (features): Sequential(
    (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (4): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (8): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (11): BatchNorm3d(64, eps=1e-05, momentum=0

<All keys matched successfully>

In [6]:
from tqdm import tqdm

cam_outputs, class_indices, logits = [], [], []
for x_batch, y_cls, y_reg, _ in tqdm(dl_va, desc="Val", leave=False):
    x_batch = x_batch.to(args.device)
    for x in x_batch:
        x = x.unsqueeze(0) # [1, C, D, H, W]
        cam_np, class_idx, logit = grad_cam(model, x, target_layer = None, class_idx = None)
        cam_outputs.append(cam_np)
        class_indices.append(class_idx)
        logits.append(logit)

cam_outputs = np.array(cam_outputs)
print('cam output shape:', cam_outputs.shape)
np.save(f'{args.output_path}/{args.dataset}/CamCAN_test.npy', cam_outputs)

df = pd.DataFrame(np.concatenate(logits, axis=0) , columns=["logit1", "logit2"])
df["class_idx"] = pd.Series(class_indices).values
display(df)
df.to_csv(f'{args.output_path}/{args.dataset}/CamCAN_test_indeces.csv')

cam output shape: (2069, 128, 128, 128)


,logit1,logit2,class_idx
0,0.497825,-1.755944,1
1,-1.486864,0.800873,0
2,2.319846,-3.743148,1
3,1.944412,-3.418749,1
4,-1.037151,0.357382,0
...,...,...,...
2064,4.139723,-6.121572,1
2065,-1.489342,0.800507,0
2066,0.548814,-1.653964,1
2067,1.671721,-2.951716,1


In [ ]:
import numpy as np

cam_np_all = np.load(f'{args.output_path}/{args.dataset}/CamCAN_test.npy')
print(cam_np_all.shape)

df_idx = pd.read_csv('/Users/yu7637xi/Desktop/Projects/models/AI-PET/results/CNN3D_visual_read_2split80-20_stratify-visual_read,site_IDEAS_Inten_Norm_20251004_022211/validation/IDEAS/CamCAN_test_indeces.csv', index_col=0)

for g in ['all', 0, 1, 'subj']:
    print(g)
    if g == 'all':
        cam_np_ave = np.mean(cam_np_all, axis=0)
    elif g == 'subj':
        cam_np_ave = cam_np_all[10]
    else:
        idx = df_idx[df_idx['class_idx']==1].index.tolist()
        cam_np_ave = np.mean(cam_np_all[idx], axis=0)

    gradcam_to_nifti(cam_np_ave, out_path= f'{args.output_path}/{args.dataset}/GradCAM_test_ave_{g}.nii', 
                 ref_nii_path='/Users/yu7637xi/Desktop/Projects/models/AI-PET/data/test/PET/50154/PET_Florbetapir_Conv,_Non-Rigid_Reg_to_Std_Img_Vox_Size,_Uniform_Res,_Inten_Norm/1940-11-04_10_32_04.0/I10422416/IDEAS_50154_20240409140129048.nii')
    
    view = plot_stat_nii(stat_path=f'{args.output_path}/{args.dataset}/GradCAM_test_ave_{g}.nii', abs_t=0.1) 
    display(view)

#### CL

In [5]:
import os
test_set = os.path.join(args.best_model_folder,'Hold-out_testing-set.csv')
print(test_set)
df = pd.read_csv(test_set, index_col=0)
tfm = args.input_path

dl_va = get_loader(df, tfm, args, batch_size=max(1, args.batch_size // 2), augment=False, shuffle=False)

/proj/berzelius-2024-156/users/x_yxiao/AI-PET/results/CNN3D_CL_2split80-20_stratify-visual_read,site_IDEAS_Inten_Norm_20251004_022211/Hold-out_testing-set.csv


In [6]:
import torch

targets_list = [t.strip() for t in args.targets.split(",") if t.strip()]
n_classes = int(df["visual_read"].dropna().nunique()) if 'visual_read' in targets_list else None
model = build_model_from_args(args, device=args.device, n_classes=n_classes)
sd = torch.load(os.path.join(args.best_model_folder, 'outer-test/checkpoints', 'best.pt'), map_location=args.device, weights_only=True)
state_dict = sd.get("model", sd) if isinstance(sd, dict) else sd
model.load_state_dict(state_dict, strict=False)

in_channels: 1, widths: (32, 64, 128, 256), pool_every: 1, dropout: 0.3, norm: batch, num_classes: 1
CNN3D(
  (features): Sequential(
    (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (4): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (8): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    (11): BatchNorm3d(64, eps=1e-05, momentum=0

<All keys matched successfully>

In [7]:
from tqdm import tqdm

cam_outputs, class_indices, logits = [], [], []
for x_batch, y_cls, y_reg, _ in tqdm(dl_va, desc="Val", leave=False):
    x_batch = x_batch.to(args.device)
    for x in x_batch:
        x = x.unsqueeze(0) # [1, C, D, H, W]
        cam_np, class_idx, logit = grad_cam(model, x, target_layer = None, class_idx = None)
        cam_outputs.append(cam_np)
        class_indices.append(class_idx)
        logits.append(logit)

cam_outputs = np.array(cam_outputs)
print('cam output shape:', cam_outputs.shape)
np.save(f'{args.output_path}/{args.dataset}/CamCAN_test.npy', cam_outputs)

cam output shape: (2069, 128, 128, 128)


FileNotFoundError: [Errno 2] No such file or directory: '/proj/berzelius-2024-156/users/x_yxiao/AI-PET/results/CNN3D_CL_2split80-20_stratify-visual_read,site_IDEAS_Inten_Norm_20251004_022211/validation/IDEAS/CamCAN_test.npy'

In [8]:
np.save(f'{args.output_path}/{args.dataset}/CamCAN_test.npy', cam_outputs)

### ADNI

#### Visual read

In [ ]:
import os
from src.data import get_transforms

test_set = os.path.join(args.proj_path, "data", f'{args.dataset}_found_scans_{args.data_suffix}_{args.targets}.csv')
print(test_set)
df = pd.read_csv(test_set, index_col=0)
tfm = get_transforms()

dl_va = get_loader(df, tfm, args, batch_size=max(1, args.batch_size // 2), augment=False, shuffle=False)

In [ ]:
import torch

targets_list = [t.strip() for t in args.targets.split(",") if t.strip()]
n_classes = int(df["visual_read"].dropna().nunique()) if 'visual_read' in targets_list else None
model = build_model_from_args(args, device=args.device, n_classes=n_classes)
sd = torch.load(os.path.join(args.best_model_folder, 'outer-test/checkpoints', 'best.pt'), map_location=args.device, weights_only=True)
state_dict = sd.get("model", sd) if isinstance(sd, dict) else sd
model.load_state_dict(state_dict, strict=False)

In [ ]:
from tqdm import tqdm

cam_outputs, class_indices, logits = [], [], []
for x_batch, y_cls, y_reg, _ in tqdm(dl_va, desc="Val", leave=False):
    x_batch = x_batch.to(args.device)
    for x in x_batch:
        x = x.unsqueeze(0) # [1, C, D, H, W]
        cam_np, class_idx, logit = grad_cam(model, x, target_layer = None, class_idx = None)
        cam_outputs.append(cam_np)
        class_indices.append(class_idx)
        logits.append(logit)

cam_outputs = np.array(cam_outputs)
print('cam output shape:', cam_outputs.shape)
np.save(f'{args.output_path}/{args.dataset}/CamCAN_test.npy', cam_outputs)

df = pd.DataFrame(np.concatenate(logits, axis=0) , columns=["logit1", "logit2"])
df["class_idx"] = pd.Series(class_indices).values
display(df)
df.to_csv(f'{args.output_path}/{args.dataset}/CamCAN_test_indeces.csv')